In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('/datasets/ud730/mnist/', one_hot=True, reshape=False)

Extracting /datasets/ud730/mnist/train-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist/train-labels-idx1-ubyte.gz
Extracting /datasets/ud730/mnist/t10k-images-idx3-ubyte.gz
Extracting /datasets/ud730/mnist/t10k-labels-idx1-ubyte.gz


In [87]:
# Parameters
learning_rate = 0.0001
epochs = 10
batch_size = 128

In [88]:
test_valid_size = 256

In [89]:
n_classes = 10
dropout = 0.75 # probability to keep units

In [90]:
# weights and bias
weights = {
    'wc1' : tf.Variable( tf.random_normal([5, 5, 1, 32]) ),
    'wc2' : tf.Variable( tf.random_normal([5, 5, 32, 64]) ),
    'wd1' : tf.Variable( tf.random_normal([7*7*64, 1024]) ), 
    'out' : tf.Variable( tf.random_normal([1024, n_classes]) )
}
biases = {
    'bc1' : tf.Variable( tf.zeros([32]) ),
    'bc2' : tf.Variable( tf.zeros([64]) ),
    'bd1' : tf.Variable( tf.zeros([1024]) ),
    'out' : tf.Variable( tf.zeros([n_classes]) ),
}

In [91]:
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d( x, W, strides=[1, strides, strides, 1], padding='SAME' )
    x = tf.nn.bias_add( x, b )
    return tf.nn.relu(x)

In [92]:
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding="SAME")

In [93]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d( x, weights['wc1'], biases['bc1'] )
    conv1 = maxpool2d(conv1, k=2)
    
    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d( conv1, weights['wc2'], biases['bc2'] )
    conv2 = maxpool2d(conv2, k=2)
    
    # Layer 3 Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add( tf.matmul(fc1, weights['wd1']), biases['bd1'] )
    fc1 = tf.nn.relu( fc1 )
    fc1 = tf.nn.dropout( fc1, dropout )
    
    # Output Layer - 1024 to 10
    out = tf.add( tf.matmul(fc1, weights['out']), biases['out'] )
    return out

## Session

In [94]:
# tf input
x = tf.placeholder(tf.float32, [None, 28,28,1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

In [95]:
# Model
logits = conv_net( x, weights, biases, keep_prob )

In [96]:
# Define loss and optimizer
cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y) )
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

In [97]:
# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean( tf.cast(correct_pred, tf.float32) )

In [98]:
init = tf.global_variables_initializer()

In [ ]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(mnist.train.num_examples//batch_size):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run( optimizer, feed_dict={x:batch_x, y:batch_y, keep_prob:dropout} )
            
            # Calculate batch loss and accuracy
            loss = sess.run( cost, feed_dict={x:batch_x, y:batch_y, keep_prob:1.} )
            valid_acc = sess.run(accuracy, feed_dict={
                x:mnist.validation.images[:test_valid_size],
                y:mnist.validation.labels[:test_valid_size],
                keep_prob:1.0
                })
        
            print('Epoch {:>2}, Batch {:>3} -'
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                      epoch + 1,
                      batch + 1,
                      loss,
                      valid_acc
                  ))
    # Calculate the Test Accuracy
    test_acc = sess.run( accuracy, feed_dict={
        x: mnist.test.images[:test_valid_size],
        y: mnist.test.labels[:test_valid_size],
        keep_prob:1.0
    } )